
# Focal length in pixels & 3D Back-Projection — Hands-on

이 노트북은 픽셀 단위 초점거리(fx, fy)의 개념을 실습으로 확인하고,
카메라 내부 파라미터 행렬 K를 만들고,
픽셀 좌표 (u, v)와 깊이 Z로부터 3D 카메라 좌표 (X, Y, Z)를 복원하는 과정을 단계별로 시연합니다.

핵심 아이디어:
- K = [[fx, 0, cx], [0, fy, cy], [0, 0, 1]]
- X = (u - cx) / fx * Z
- Y = (v - cy) / fy * Z
- Z = Z

픽셀 단위 초점거리 구하기 (두 관점):
1) FOV 관점: 가로폭 W, 수평 FOV(theta_x)가 주어졌을 때
   fx = (W/2) / tan(theta_x/2)

2) 센서 관점: 광학 초점거리 f_mm(mm), 픽셀피치 p(mm/px)가 주어졌을 때
   f_px = f_mm / p


## 0) 준비: 라이브러리 & 헬퍼 함수

In [ ]:

import numpy as np
import pandas as pd
from math import tan, radians

# “이미지 크기(W×H)와 수평 시야각(FOV)만 알 때” 카메라 내부파라미터를 픽셀 단위로 만들어주는 간단 공식
def intrinsics_from_fov(width_px: int, height_px: int, fov_x_deg: float):
    """Compute fx, fy, cx, cy and K from image width/height and horizontal FOV.
    Assumes square pixels -> fy = fx."""
    # 1) 수평 FOV(도) -> 라디안 변환 후, 삼각비로 fx(픽셀 단위 초점거리) 계산
    # → 화면 반쪽(픽셀)과 FOV의 반각으로 픽셀 단위 초점거리를 구하는 기본 삼각비.
    fx = (width_px / 2.0) / tan(radians(fov_x_deg) / 2.0)
    # → 픽셀을 정사각형으로 가정(가로/세로 픽셀 크기가 같음)
    fy = fx
    # → **주점(Principal Point)**을 이미지의 정중앙으로 둠(간단한 기본 가정).
    cx = width_px / 2.0
    cy = height_px / 2.0
    K = np.array([[fx, 0,  cx],
                  [0,  fy, cy],
                  [0,  0,   1]], dtype=np.float32)
    return fx, fy, cx, cy, K

# 이 함수는 **광학 초점거리(㎜)**를 **픽셀 단위 초점거리(px)**로 바꿔줍니다.
# 이후 3D↔2D 투영/역투영, 포인트클라우드 복원, 깊이와의 결합 등에 바로 쓰입니다.
def focal_px_from_sensor(f_mm: float, pixel_pitch_mm: float):
    """Convert optical focal length (mm) and pixel pitch (mm/px) to focal length in pixels."""
    # f_mm: 렌즈 초점거리(밀리미터)                        # f_mm의 단위: mm
    # pixel_pitch_mm: 픽셀 한 칸의 물리 크기(밀리미터/픽셀)  # pixel_pitch_mm의 단위: mm/px
    return f_mm / pixel_pitch_mm                        # f_mm / pixel_pitch_mm의 단위는 (mm) / (mm/px) = px
                                                        # → 결과가 픽셀 단위 초점거리가 됩니다.

def backproject(u, v, Z, fx, fy, cx, cy):
    """Back-project a pixel (u,v) with depth Z (meters) to camera coordinates (X,Y,Z)."""
    X = (u - cx) * Z / fx
    Y = (v - cy) * Z / fy
    return float(X), float(Y), float(Z)

def scale_intrinsics(fx, fy, cx, cy, sx, sy):
    """Scale intrinsics consistently when the image is resized by factors (sx, sy)."""
    fx2 = fx * sx
    fy2 = fy * sy
    cx2 = cx * sx
    cy2 = cy * sy
    return fx2, fy2, cx2, cy2

print('Helpers loaded.')


Helpers loaded.



## 1) 예시 A — FOV로부터 fx, fy, cx, cy 구하기

입력: 이미지 해상도 W x H, 수평 FOV (degrees)


In [4]:

# ---- 파라미터를 바꿔보세요 ----
W, H = 1280, 720
fov_x_deg = 60.0  # 수평 FOV (degrees)

fx_A, fy_A, cx_A, cy_A, K_A = intrinsics_from_fov(W, H, fov_x_deg)
print(f"[A] Image: {W}x{H}px, FOV_x: {fov_x_deg}°")
print(f"    fx={fx_A:.3f}px, fy={fy_A:.3f}px, cx={cx_A:.1f}px, cy={cy_A:.1f}px")
print("    K =\n", K_A)


[A] Image: 1280x720px, FOV_x: 60.0°
    fx=1108.513px, fy=1108.513px, cx=640.0px, cy=360.0px
    K =
 [[1.1085126e+03 0.0000000e+00 6.4000000e+02]
 [0.0000000e+00 1.1085126e+03 3.6000000e+02]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00]]


## ***** Fx 값은 3D를 2D로 옮길 때의 배율을 결정하게 됨 *****
  ![alt text](111.png)
  ![alt text](222.png)


## 2) 예시 B — 센서 파라미터(mm, 픽셀피치)에서 f_px 구하기

입력: 광학 초점거리 f_mm (mm), 픽셀 피치 p (mm/px)


In [5]:

# ---- 파라미터를 바꿔보세요 ----
f_mm = 4.0              # mm (예: 스마트폰 광각 렌즈)
pixel_pitch_um = 1.12   # μm (마이크로미터) per pixel
pixel_pitch_mm = pixel_pitch_um * 1e-3  # μm -> mm

fx_B = focal_px_from_sensor(f_mm, pixel_pitch_mm)
fy_B = fx_B  # 정사각 픽셀 가정
cx_B, cy_B = W/2.0, H/2.0
K_B = np.array([[fx_B, 0, cx_B],
                [0, fy_B, cy_B],
                [0, 0, 1]], dtype=np.float32)

print(f"[B] f_mm = {f_mm} mm, pixel_pitch = {pixel_pitch_um} μm -> fx ≈ {fx_B:.1f}px")
print("    K =\n", K_B)


[B] f_mm = 4.0 mm, pixel_pitch = 1.12 μm -> fx ≈ 3571.4px
    K =
 [[3.5714285e+03 0.0000000e+00 6.4000000e+02]
 [0.0000000e+00 3.5714285e+03 3.6000000e+02]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00]]


## ***** 상기 K값을 구하는 두 개의 방식의 결과는 다른가?? *****
  ![alt text](111-1.png)
  ![alt text](222-1.png)

## ***** DepthPro 에서는 시야각을 안줬는데 초점거리를 어떻게 알아냈는가??? *****
  ![alt text](111-2.png)
  ![alt text](222-2.png)


## 3) 예시 A의 K로 3D 역투영

픽셀 좌표 (u,v)와 깊이 Z가 있을 때 (X,Y,Z)를 계산합니다.


In [ ]:

# ---- 파라미터를 바꿔보세요 ----
u, v = 800.0, 350.0  # 픽셀 좌표
Z = 2.5              # 깊이 (meters)

X_A, Y_A, Z_A = backproject(u, v, Z, fx_A, fy_A, cx_A, cy_A)
print(f"[Back-Projection A] (u,v)=({u},{v}), Z={Z} m -> (X,Y,Z)=({X_A:.4f}, {Y_A:.4f}, {Z_A:.4f}) m")



## 4) 리사이즈 시 내부 파라미터 스케일링

이미지를 0.5x로 줄이면 fx, fy, cx, cy도 동일한 비율로 스케일해야 3D 결과가 불변입니다.


In [ ]:

# ---- 파라미터를 바꿔보세요 ----
sx, sy = 0.5, 0.5             # 가로/세로 스케일
W2, H2 = int(W * sx), int(H * sy)

fx_A2, fy_A2, cx_A2, cy_A2 = scale_intrinsics(fx_A, fy_A, cx_A, cy_A, sx, sy)

# 리사이즈 후 해당 픽셀의 좌표
u2, v2 = u * sx, v * sy
X_A2, Y_A2, Z_A2 = backproject(u2, v2, Z, fx_A2, fy_A2, cx_A2, cy_A2)

print(f"[Resize] {W}x{H}px -> {W2}x{H2}px (sx={sx}, sy={sy})")
print(f"         fx={fx_A2:.3f}, fy={fy_A2:.3f}, cx={cx_A2:.1f}, cy={cy_A2:.1f}")
print(f"         (u2,v2)=({u2:.1f},{v2:.1f}), Z={Z} m -> (X,Y,Z)=({X_A2:.4f}, {Y_A2:.4f}, {Z_A2:.4f}) m")
print('Note: (X,Y,Z)가 앞 단계와 동일해야 올바르게 스케일된 것입니다.')



## 5) 요약 테이블 (비교)


In [ ]:

summary = pd.DataFrame([
    {"Case": "A (from FOV)",          "W": W,  "H": H,  "fx": fx_A,  "fy": fy_A,  "cx": cx_A,  "cy": cy_A,
     "u": u, "v": v, "Z(m)": Z, "X(m)": X_A, "Y(m)": Y_A},
    {"Case": "A resized (0.5x)",      "W": W2, "H": H2, "fx": fx_A2, "fy": fy_A2, "cx": cx_A2, "cy": cy_A2,
     "u": u2, "v": v2, "Z(m)": Z_A2, "X(m)": X_A2, "Y(m)": Y_A2},
    {"Case": "B (mm->px via pitch)",  "W": W,  "H": H,  "fx": fx_B,  "fy": fy_B,  "cx": cx_B,  "cy": cy_B,
     "u": np.nan, "v": np.nan, "Z(m)": np.nan, "X(m)": np.nan, "Y(m)": np.nan},
])
summary



## 6) (옵션) DepthPro 출력과 결합

`result["focallength_px"]`가 스칼라라면 fx=fy=focal_px 로, 2-튜플이라면 (fx, fy)로 사용하세요.
리사이즈/크롭을 했다면 fx, fy, cx, cy도 동일한 규칙으로 조정해야 합니다.


In [ ]:

# 예시: DepthPro에서 받은 초점거리(px)를 사용해 K 만들기
# focal_px_depthpro = 1100.0                     # (1) 스칼라 케이스
# fx_dp = fy_dp = float(focal_px_depthpro)

# 또는 (fx, fy)로 오는 경우
# fx_dp, fy_dp = 1120.0, 1105.0                  # (2) 튜플 케이스

# 이미지 중심 (보통 W/2, H/2)
# cx_dp, cy_dp = W/2.0, H/2.0
# K_dp = np.array([[fx_dp, 0, cx_dp],
#                  [0, fy_dp, cy_dp],
#                  [0, 0, 1]], dtype=np.float32)
# K_dp
